# Knapsack

$$
\begin{align}
    \text{max} & \quad \sum_{i \in I} c_{i} x_{i}\\
    \text{s.t.} & \quad \sum_{i \in I} w_{i} x_{i} \leq Q\\
    & x_{i} \in \{0, 1\} & \forall i \in I
\end{align}
$$

In [1]:
import pyomo.environ as pyo

In [3]:
# Define input data
costs = {
    "A": 4,
    "B": 2,
    "C": 10,
    "D": 5,
    "E": 6,
}

weights = {
    "A": 3,
    "B": 2,
    "C": 7,
    "D": 4,
    "E": 5,
}

capacity = 11

In [5]:
# Instantiate model as a Pyomo ConcreteModel
model = pyo.ConcreteModel()

In [12]:
#costs.keys() # Get the list of items

In [9]:
# Sets
model.I = pyo.Set(initialize=costs.keys())  # Items

In [13]:
#model.I.display() # Display the set of items

In [14]:
# Parameters
model.c = pyo.Param(model.I, initialize=costs)  # Costs
model.w = pyo.Param(model.I, initialize=weights)  # Weights
model.Q = pyo.Param(initialize=capacity)  # Capacity

In [16]:
#model.c.display() # Display the costs

In [17]:
# Variables
model.x = pyo.Var(model.I, within=pyo.Binary)  # Selection of items

In [19]:
#model.x.display() # Display the variables

In [20]:
# Constraints
def capacity_cstr(model):
    return sum(model.w[i] * model.x[i] for i in model.I) <= model.Q


model.cap_cstr = pyo.Constraint(rule=capacity_cstr)  # Capacity constraint

In [21]:
model.cap_cstr.pprint() # Display the capacity constraint

cap_cstr : Size=1, Index=None, Active=True
    Key  : Lower : Body                                       : Upper : Active
    None :  -Inf : 3*x[A] + 2*x[B] + 7*x[C] + 4*x[D] + 5*x[E] :  11.0 :   True


In [22]:
# Objective
def obj(model):
    return sum(model.c[i] * model.x[i] for i in model.I)


model.obj = pyo.Objective(rule=obj, sense=pyo.maximize)  # Objective function

In [23]:
model.obj.pprint() # Display the objective function

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 4*x[A] + 2*x[B] + 10*x[C] + 5*x[D] + 6*x[E]


In [24]:
# Instantiate Highs persistent solver 
solver = pyo.SolverFactory("appsi_highs")

In [26]:
# Apply method solve
result = solver.solve(model)
print(result)  # Print the result


Problem: 
- Lower bound: 15.0
  Upper bound: 15.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 0
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Termination message: TerminationCondition.optimal
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [27]:
# Create output dictionary from variables
model.x.extract_values()

{'A': -0.0, 'B': 0.0, 'C': 1.0, 'D': 1.0, 'E': 0.0}